In [1]:
sc

Intitializing Scala interpreter ...

Spark Web UI available at http://172.16.8.92:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1584605995158)
SparkSession available as 'spark'


res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@1cb187c


In [2]:
val datapath = "/Users/soda/Downloads/backUp/Notebooks/AdvancedAnalyticswithSpark/data/"

datapath: String = /Users/soda/Downloads/backUp/Notebooks/AdvancedAnalyticswithSpark/data/


In [3]:
val dataWithoutHeader = spark.read.
    option("inferSchema",true).
    option("header",false).
    csv(datapath+"kddcup.data")

dataWithoutHeader: org.apache.spark.sql.DataFrame = [_c0: int, _c1: string ... 40 more fields]


In [4]:
val data = dataWithoutHeader.toDF(
  "duration", "protocol_type", "service", "flag",
  "src_bytes", "dst_bytes", "land", "wrong_fragment", "urgent",
  "hot", "num_failed_logins", "logged_in", "num_compromised",
  "root_shell", "su_attempted", "num_root", "num_file_creations",
  "num_shells", "num_access_files", "num_outbound_cmds",
  "is_host_login", "is_guest_login", "count", "srv_count",
  "serror_rate", "srv_serror_rate", "rerror_rate", "srv_rerror_rate",
  "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
  "dst_host_count", "dst_host_srv_count",
  "dst_host_same_srv_rate", "dst_host_diff_srv_rate",
  "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
  "dst_host_serror_rate", "dst_host_srv_serror_rate",
  "dst_host_rerror_rate", "dst_host_srv_rerror_rate",
  "label")

data: org.apache.spark.sql.DataFrame = [duration: int, protocol_type: string ... 40 more fields]


In [5]:
println(data.columns.length,data.count())

(42,4898431)


In [11]:
data.select("label").groupBy("label").count().orderBy($"count".desc).show(25)

+----------------+-------+
|           label|  count|
+----------------+-------+
|          smurf.|2807886|
|        neptune.|1072017|
|         normal.| 972781|
|          satan.|  15892|
|        ipsweep.|  12481|
|      portsweep.|  10413|
|           nmap.|   2316|
|           back.|   2203|
|    warezclient.|   1020|
|       teardrop.|    979|
|            pod.|    264|
|   guess_passwd.|     53|
|buffer_overflow.|     30|
|           land.|     21|
|    warezmaster.|     20|
|           imap.|     12|
|        rootkit.|     10|
|     loadmodule.|      9|
|      ftp_write.|      8|
|       multihop.|      7|
|            phf.|      4|
|           perl.|      3|
|            spy.|      2|
+----------------+-------+



In [123]:
val Array(trainData, testData) = data.randomSplit(Array(0.8, 0.2))

trainData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [duration: int, protocol_type: string ... 40 more fields]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [duration: int, protocol_type: string ... 40 more fields]


In [124]:
val dataSample = testData.toDF()

dataSample: org.apache.spark.sql.DataFrame = [duration: int, protocol_type: string ... 40 more fields]


In [125]:
// dataSample.select("label").groupBy("label").count().orderBy($"count".desc).show(25)

+----------------+------+
|           label| count|
+----------------+------+
|          smurf.|561923|
|        neptune.|214735|
|         normal.|194824|
|          satan.|  3238|
|        ipsweep.|  2590|
|      portsweep.|  2140|
|           nmap.|   454|
|           back.|   413|
|       teardrop.|   193|
|    warezclient.|   186|
|            pod.|    53|
|   guess_passwd.|    12|
|buffer_overflow.|    10|
|    warezmaster.|     3|
|        rootkit.|     2|
|     loadmodule.|     2|
|           land.|     2|
|           imap.|     1|
|           perl.|     1|
|      ftp_write.|     1|
|            phf.|     1|
+----------------+------+



In [12]:
data.first

res6: org.apache.spark.sql.Row = [0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,normal.]


In [126]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.clustering.{KMeans,KMeansModel}
import org.apache.spark.ml.feature.VectorAssembler

val numericOnly = dataSample.drop("protocol_type","service","flag").cache()
val assembler = new VectorAssembler().
    setInputCols(numericOnly.columns.filter(_!="label")).
    setOutputCol("featureVector")

val kmeans = new KMeans().
    setPredictionCol("cluster").
    setFeaturesCol("featureVector")

val pipeline = new Pipeline().setStages(Array(assembler,kmeans))
val pipelineModel = pipeline.fit(numericOnly)
val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.feature.VectorAssembler
numericOnly: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [duration: int, src_bytes: int ... 37 more fields]
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_56c71104e9a0
kmeans: org.apache.spark.ml.clustering.KMeans = kmeans_641090b27e21
pipeline: org.apache.spark.ml.Pipeline = pipeline_33d94be62a20
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_33d94be62a20
kmeansModel: org.apache.spark.ml.clustering.KMeansModel = kmeans_641090b27e21


In [127]:
kmeansModel.clusterCenters.foreach(println)

[49.38001168454184,1501.835438624038,1049.7732933788616,2.0391870576875822E-6,6.372459555273694E-4,2.1411464105719613E-5,0.0118068930640111,3.2626992923001316E-5,0.14353022024239817,0.007641853498684214,7.137154701906538E-5,3.670536703837648E-5,0.012652136099422603,0.0011541798746511716,7.544992113444054E-5,0.0010175543417861035,0.0,1.0195935288437911E-6,8.014005136712198E-4,334.8298991723959,295.0964443714869,0.17803137901044383,0.17809188169044526,0.05756938079065412,0.057643026031242356,0.7897316939628652,0.02127689815178726,0.02838519835682347,232.98639658313817,189.12281106014277,0.7532675627534575,0.03089434665976848,0.6050103437762813,0.006458625404396378,0.17817002333849663,0.17795975256504307,0.057846822385788074,0.057605433617834406]
[36071.0,1.379963888E9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,77.0,1.0,0.01,0.12,0.06,0.0,0.0,0.0,0.06,1.0]


In [128]:
val withCluster = pipelineModel.transform(numericOnly)

withCluster: org.apache.spark.sql.DataFrame = [duration: int, src_bytes: int ... 39 more fields]


In [129]:
withCluster.select("cluster","label").
    groupBy("cluster","label").count().
    orderBy($"cluster",$"count".desc).show(25)

+-------+----------------+------+
|cluster|           label| count|
+-------+----------------+------+
|      0|          smurf.|561923|
|      0|        neptune.|214735|
|      0|         normal.|194824|
|      0|          satan.|  3238|
|      0|        ipsweep.|  2590|
|      0|      portsweep.|  2139|
|      0|           nmap.|   454|
|      0|           back.|   413|
|      0|       teardrop.|   193|
|      0|    warezclient.|   186|
|      0|            pod.|    53|
|      0|   guess_passwd.|    12|
|      0|buffer_overflow.|    10|
|      0|    warezmaster.|     3|
|      0|           land.|     2|
|      0|        rootkit.|     2|
|      0|     loadmodule.|     2|
|      0|            phf.|     1|
|      0|           perl.|     1|
|      0|           imap.|     1|
|      0|      ftp_write.|     1|
|      1|      portsweep.|     1|
+-------+----------------+------+



In [139]:
//Sum of Square
import scala.util.Random
import org.apache.spark.sql.DataFrame

def clusteringScore0(data:DataFrame,k:Int):Double = {
    val assembler = new VectorAssembler().
        setInputCols(data.columns.filter(_!="label")).
        setOutputCol("featureVector")
    val kmeans = new KMeans().
        setSeed(Random.nextLong()).
        setK(k).
        setPredictionCol("cluster").
        setFeaturesCol("featureVector")
    val pipeline = new Pipeline().setStages(Array(assembler,kmeans))
    val kmeansModel = pipeline.fit(data).stages.last.asInstanceOf[KMeansModel]
    kmeansModel.computeCost(assembler.transform(data))/data.count()
}

import scala.util.Random
import org.apache.spark.sql.DataFrame
clusteringScore0: (data: org.apache.spark.sql.DataFrame, k: Int)Double


In [140]:
(20 to 100 by 20).map(k => (k, clusteringScore0(numericOnly, k))).
  foreach(println)

(20,8341731.132478421)
(40,7127933.177598517)
(60,3542366.9420348885)
(80,3434540.5904618637)
(100,3400234.45255818)


# Stratified Sampling

## randomSplit

In [22]:
val obs = spark.read.option("inferSchema",true).csv("/Users/soda/Downloads/iris.data")

obs: org.apache.spark.sql.DataFrame = [_c0: double, _c1: double ... 3 more fields]


In [36]:
val Array(trainData, testData) = obs.randomSplit(Array(0.9, 0.1))

trainData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_c0: double, _c1: double ... 3 more fields]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [_c0: double, _c1: double ... 3 more fields]


In [43]:
trainData.toDF().groupBy("_c4").count().select($"count"/trainData.count()).show()

+------------------+
|     (count / 132)|
+------------------+
|0.3409090909090909|
|0.3181818181818182|
|0.3409090909090909|
+------------------+



In [117]:
trainData.toDF().show(3)

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|4.3|3.0|1.1|0.1|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.4|3.0|1.3|0.2|Iris-setosa|
+---+---+---+---+-----------+
only showing top 3 rows



## sampleBy
http://allaboutscala.com/big-data/spark/

In [69]:
// https://spark.apache.org/docs/1.6.2/api/java/org/apache/spark/sql/DataFrameStatFunctions.html
val fraction = Map("Iris-virginica" -> 0.1,
                  "Iris-setosa" -> 0.1,
                  "Iris-versicolor" -> 0.1)

fraction: scala.collection.immutable.Map[String,Double] = Map(Iris-virginica -> 0.1, Iris-setosa -> 0.1, Iris-versicolor -> 0.1)


In [104]:
obs.stat.sampleBy("_c4",fraction,15L).groupBy("_c4").count().show()

+---------------+-----+
|            _c4|count|
+---------------+-----+
| Iris-virginica|    6|
|    Iris-setosa|    6|
|Iris-versicolor|    5|
+---------------+-----+



https://stackoverflow.com/questions/41516805/how-to-select-a-same-size-stratified-sample-from-a-dataframe-in-apache-spark

In [8]:
data.printSchema

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file_creations: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- num_outbound_cmds: integer (nullable = true)
 |-- is_host_login: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true

In [16]:
import org.apache.spark.sql.functions.row_number
import org.apache.spark.sql.expressions.Window


val sampling = data.withColumn("row_num",
              row_number().over(Window.partitionBy($"label").orderBy($"duration"))).
              where(col("row_num") <= 5).count()

import org.apache.spark.sql.functions.row_number
import org.apache.spark.sql.expressions.Window
sampling: org.apache.spark.sql.DataFrame = [duration: int, protocol_type: string ... 41 more fields]


In [20]:
println(sampling.count(),data.count())

(4898431,4898431)


res15: Long = 109


5.8　特征的规范化

In [144]:
import org.apache.spark.ml.feature.StandardScaler

def clusteringScore2(data: DataFrame, k: Int): Double = {
  val assembler = new VectorAssembler().
    setInputCols(data.columns.filter(_ != "label")).
    setOutputCol("featureVector")
  val scaler = new StandardScaler()
    .setInputCol("featureVector")
    .setOutputCol("scaledFeatureVector")
    .setWithStd(true)
    .setWithMean(false)
  val kmeans = new KMeans().
    setSeed(Random.nextLong()).
    setK(k).
    setPredictionCol("cluster").
    setFeaturesCol("scaledFeatureVector").
    setMaxIter(40).
    setTol(1.0e-5)
  val pipeline = new Pipeline().setStages(Array(assembler, scaler, kmeans))
  val pipelineModel = pipeline.fit(data)
  val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
  kmeansModel.computeCost(pipelineModel.transform(data)) / data.count()
}

import org.apache.spark.ml.feature.StandardScaler
clusteringScore2: (data: org.apache.spark.sql.DataFrame, k: Int)Double


In [145]:
(60 to 180 by 30).map(k => (k,clusteringScore2(numericOnly,k))).
    foreach(println)

(60,1.0645668998700453)
(90,0.6114058873865597)
(120,0.41312963763016175)
(150,0.3260755679793301)
(180,0.2655638216810723)


5.9　类别型变量

In [146]:
import org.apache.spark.ml.feature.{OneHotEncoder,StringIndexer}

def oneHotPipeline(inputCol:String):(Pipeline,String) = {
    val indexer = new StringIndexer().
        setInputCol(inputCol).
        setOutputCol(inputCol+"_indexed")
    
    val encoder = new OneHotEncoder().
        setInputCol(inputCol+"_indexed").
        setOutputCol(inputCol+"_vec")
    
    val pipeline = new Pipeline().setStages(Array(indexer,encoder))
    (pipeline,inputCol+"_vec")
}

import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}
oneHotPipeline: (inputCol: String)(org.apache.spark.ml.Pipeline, String)


In [148]:
def entropy(counts: Iterable[Int]): Double = {
  val values = counts.filter(_ > 0)
  val n = values.map(_.toDouble).sum
  values.map { v =>
    val p = v / n
    -p * math.log(p)
  }.sum
}

entropy: (counts: Iterable[Int])Double


In [ ]:
val clusterLabel = pipelineModel.transform(data).
  select("cluster", "label").as[(Int, String)] 

val weightedClusterEntropy = clusterLabel.
  groupByKey { case (cluster, _) => cluster }. 
  mapGroups { case (_, clusterLabels) =>
    val labels = clusterLabels.map { case (_, label) => label }.toSeq
    val labelCounts = labels.groupBy(identity).values.map(_.size) 
    labels.size * entropy(labelCounts)
  }.collect()

weightedClusterEntropy.sum / data.count() 